# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *


In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/3_cond_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [9]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210105_223141_cgan_b_sigmafloat_m2', '20201220_073…

In [10]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210106_063353_cgan_b_sigmafloat_3classes_m2


## Plot Losses

In [11]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [12]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
46851,46851.0,19.0,0.168684,0.183741,0.352425,4.935838,82.543266,77.607430,-2.584685,80.224739,-2.144699,3.622783,-2.949704,-4.918343,0.145587
46852,46852.0,19.0,0.250921,0.145715,0.396636,3.137248,83.952705,80.815460,-2.288500,83.804726,-2.905500,2.412682,-3.941113,-3.040988,0.148120
46853,46853.0,19.0,0.096086,0.216844,0.312930,4.911223,85.416000,80.504776,-2.661388,80.718071,-2.215647,3.942012,-2.329053,-4.894675,0.151546
46854,46854.0,19.0,0.203955,0.148923,0.352877,4.981210,86.617310,81.636101,-2.822162,80.153671,-2.650482,3.055762,-3.155040,-4.966065,0.145935
46855,46855.0,19.0,0.335375,0.108234,0.443608,1.639629,83.787636,82.148010,-2.468554,82.373161,-2.870477,1.885889,-3.891896,-1.190240,0.148447
46856,46856.0,19.0,0.093653,0.769583,0.863236,11.564919,91.679466,80.114548,-2.136315,79.725777,-2.727677,5.785879,-0.480845,-11.564903,0.146896
46857,46857.0,19.0,1.770010,0.138800,1.908810,0.541530,81.129532,80.588005,-2.236904,80.473999,-2.076853,-1.488048,-9.426304,1.012281,0.147418
46858,46858.0,19.0,0.121028,2.700461,2.821489,15.246496,95.528648,80.282150,-2.275721,79.656204,-1.970463,9.602232,2.635586,-15.246496,0.179366
46859,46859.0,19.0,2.145844,0.212428,2.358271,6.299715,87.668808,81.369095,-2.258996,82.194641,-1.686566,-1.820773,-13.463179,-6.293508,0.146229
46860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.087486,-1.549680,NaN,NaN,NaN,NaN


In [13]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [14]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

-2.5932109355926514


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
45426,45426.0,19.0,0.124158,0.334909,0.459067,7.053195,87.444672,80.391479,-1.919426,84.098236,-3.069964,5.456960,-1.952571,-7.051473,0.148149
43903,43903.0,18.0,0.103274,0.086592,0.189866,6.066842,86.711143,80.644302,-2.426552,79.711014,-3.065076,4.030417,-5.172444,-6.058811,0.150269
43912,43912.0,18.0,0.527609,0.134526,0.662135,2.220684,81.487686,79.267006,-2.886776,80.316368,-3.052977,1.309558,-5.904954,-1.992412,0.147286
42461,42461.0,18.0,0.101719,0.174227,0.275946,3.791237,85.088478,81.297241,-2.809081,82.185303,-3.043693,5.145314,-2.660257,-3.745379,0.142579
43609,43609.0,18.0,0.304383,0.122767,0.427150,8.386666,90.755363,82.368698,-2.848412,81.615501,-3.028185,2.394120,-8.286564,-8.385876,0.147680
43539,43539.0,18.0,0.096462,0.455988,0.552450,7.850522,84.109421,76.258896,-2.717918,80.004272,-3.025814,6.562231,-1.207248,-7.849695,0.146281
39841,39841.0,17.0,0.091978,0.643901,0.735880,8.563307,91.017151,82.453842,-2.706899,79.570442,-3.025514,6.692666,-0.666194,-8.563000,0.147708
43908,43908.0,18.0,0.097274,0.127432,0.224706,4.915064,88.603867,83.688805,-2.442995,81.185036,-3.025161,3.710335,-3.865843,-4.890600,0.148891
40300,40300.0,17.0,0.103063,0.128418,0.231481,4.356101,83.130440,78.774338,-2.418812,80.711411,-3.023407,3.169896,-3.897948,-4.332230,0.160981
26162,26162.0,11.0,0.270071,0.238899,0.508971,3.613834,85.708023,82.094185,-2.908622,81.374084,-3.022778,2.410331,-2.546690,-3.562389,0.145442


In [15]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

## Read stored chi-squares for images

In [16]:

df_list=[]
labels_lst=[0,1,2]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(label)))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1c']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list



In [17]:

def f_plot_hist_spec(df,class_labels,steps_list,bkg_dict,plot_type):

    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for class_label in class_labels:
        df=df[df.step.isin(steps_list)]
        print(df.shape)
        suffix='_%s'%(class_label)
        dict_bkg=bkg_dict[str(class_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(class_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']

                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
                yerr=np.sqrt((yerr2**2)/(y1**2) + (yerr1**2) * (yerr2**2/(yerr1**4)))/np.sqrt(row['num_imgs'+suffix])

#                 plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            
                plt.legend()
            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,:,:]
                f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
    
    if plot_type in ['hist','spec']:     
        plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
        plt.yscale('log')

# f_plot_hist_spec(df_merged,labels_lst,best_step,bkgnd_dict,'hist')

### Read validation data

In [30]:
np.load(fname,mmap_mode='r+')[:10].shape


OSError: [Errno 38] Function not implemented

In [18]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_list=[0.5,0.8,1.1]
labels_lst=[0,1,2]
bkgnd_dict={}
num_bkgnd=2000

for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_list[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r+')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(label)]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy


In [27]:
df_merged.columns

Index(['epoch', 'step', 'img_type', 'fname_0', 'chi_1a_0', 'chi_1b_0',
       'chi_1c_0', 'chi_1_0', 'chi_2_0', 'chi_imgvar_0', 'chi_spec1_0',
       'chi_spec2_0', 'chi_spec3_0', 'num_imgs_0', 'num_large_0',
       'num_vlarge_0', 'hist_val_0', 'hist_err_0', 'hist_bin_centers_0',
       'spec_val_0', 'spec_sdev_0', 'label', 'fname_1', 'chi_1a_1', 'chi_1b_1',
       'chi_1c_1', 'chi_1_1', 'chi_2_1', 'chi_imgvar_1', 'chi_spec1_1',
       'chi_spec2_1', 'chi_spec3_1', 'num_imgs_1', 'num_large_1',
       'num_vlarge_1', 'hist_val_1', 'hist_err_1', 'hist_bin_centers_1',
       'spec_val_1', 'spec_sdev_1', 'fname_2', 'chi_1a_2', 'chi_1b_2',
       'chi_1c_2', 'chi_1_2', 'chi_2_2', 'chi_imgvar_2', 'chi_spec1_2',
       'chi_spec2_2', 'chi_spec3_2', 'num_imgs_2', 'num_large_2',
       'num_vlarge_2', 'hist_val_2', 'hist_err_2', 'hist_bin_centers_2',
       'spec_val_2', 'spec_sdev_2', 'sum_chi_1', 'sum_chi_spec1',
       'sum_chi_1c'],
      dtype='object')

In [28]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',labels_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1c']:
            for i in labels_lst: chi_cols.append(j+'_'+str(i))
    else: ## Add chi-square for specific label
        assert label in labels_lst, "label %s is not in %s"%(label,labels_lst)
        for j in ['chi_1','chi_spec1','chi_1c']: chi_cols.append(j+'_'+str(label))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=-1,use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [20]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),label=ToggleButtons(options=['all']+labels_lst),
labels_lst=fixed(labels_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [21]:
df_sliced=w.result
# df_sliced

In [29]:
best_step=[]
best_step.append(f_slice_merged_df(df_merged,cutoff=0.8,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None]).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.8,sort_col='sum_chi_spec1',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None]).step.values)
best_step.append(f_slice_merged_df(df_merged,cutoff=0.8,sort_col='sum_chi_1c',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None]).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[21330 33770 35470 35620 42130 42760 43810 44690]


In [ ]:
# best_step=[6176]
# best_step=np.arange(40130,40135).astype(int)

In [30]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 61)
[(9, 21330), (14, 33770), (15, 35470), (15, 35620), (17, 42130), (18, 42760), (18, 43810), (19, 44690)]


In [31]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]

df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1c
2132,9-21330,0.183690,0.665152,0.023082
3376,14-33770,0.087521,2.312260,0.008725
3546,15-35470,0.224956,0.637975,0.031722
3561,15-35620,0.409513,3.813608,0.004278
4212,17-42130,0.099423,1.977167,0.024664
4275,18-42760,0.405523,8.544935,0.004760
4380,18-43810,0.104425,3.359302,0.016157
4468,19-44690,0.098553,1.457726,0.028170


In [32]:
interact_manual(f_plot_hist_spec,df=fixed(df_merged),
                class_labels=SelectMultiple(options=labels_lst),
                steps_list=SelectMultiple(options=best_step),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='class_labels', options=(0, 1, 2), value=()), SelectMultiple(…

<function __main__.f_plot_hist_spec(df, class_labels, steps_list, bkg_dict, plot_type)>